In [1]:
!pip install googletrans==3.1.0a0 -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 KB 5.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import re
from googletrans import Translator
translator = Translator()

In [32]:
df = pd.read_csv('/content/goemotions_subset_train_new_part1.csv')
df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,text,three_point,marathi_text
0,0,0,13178,"This is so hard to sit through, I can't believ...",-1,"हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर व..."
1,1,1,39432,Because idiots in this sub care too much about...,-1,कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर...
2,3,3,30127,Water is not a human rights. That's absurd.,-1,पाणी हा मानवी हक्क नाही. ते मूर्खपणाचे आहे.
3,4,4,10283,The thin side and endlines are really throwing...,-1,पातळ बाजू आणि अंतरेषा खरोखर मला फेकून देत आहेत.
4,5,5,2930,If there is no goal on this powerplay it will ...,-1,या पॉवरप्लेवर कोणतेही ध्येय नसल्यास निराश न हो...


In [33]:
df.drop(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'text'], axis=1, inplace=True)
dict1 = {'marathi_text': 'tweet', 'three_point': 'label'}
df.rename(columns=dict1, inplace=True)
df = df.iloc[:, [1, 0]]

In [34]:
df.head()

,tweet,label
0,"हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर व...",-1
1,कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर...,-1
2,पाणी हा मानवी हक्क नाही. ते मूर्खपणाचे आहे.,-1
3,पातळ बाजू आणि अंतरेषा खरोखर मला फेकून देत आहेत.,-1
4,या पॉवरप्लेवर कोणतेही ध्येय नसल्यास निराश न हो...,-1


In [35]:
# for backtranslation, convert all the goemotions comments in the df to English
df['english_translated'] = ""

In [36]:
df.head()

,tweet,label,english_translated
0,"हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर व...",-1,
1,कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर...,-1,
2,पाणी हा मानवी हक्क नाही. ते मूर्खपणाचे आहे.,-1,
3,पातळ बाजू आणि अंतरेषा खरोखर मला फेकून देत आहेत.,-1,
4,या पॉवरप्लेवर कोणतेही ध्येय नसल्यास निराश न हो...,-1,


In [37]:
# data preprocessing:
# removing non marathi text 
import re
def remove_non_marathi_goemotions(text):
    i= re.sub(r"http\S+", "", text)
    i= re.sub(r"pic\S+", "", i)
    i= re.sub(r"#", "", i)
    i= re.sub(r"lokmat\S+", "", i)
    i= re.sub(r"@\S+", "", i)
    i= re.sub(r"fb\S+", "", i)
    i= re.sub(r"twitter\S+", "", i)
    i= re.sub(r"bit\S+", "", i)    
    # i= re.sub(r"['\"!-./,;\[\]?]", "", i)  # during augmenting, keep the necessary punctuations [.?!'"] in the sentence.  
    i= re.sub(r"[\[\]]", "", i)    
    i= re.sub(r'[a-zA-Z0-9]', "", i)    
    i=re.split(r'[\s\n]+', i.strip())
    return(' '.join(i))

In [38]:
df['tweet'][205]

'[NAME] [NAME] हा कन्सोलच्या 72 तासांसारखा आहे अर्थातच आम्हाला तो wtf<3 आवडतो'

In [39]:
remove_non_marathi_goemotions(df['tweet'][205])

'हा कन्सोलच्या तासांसारखा आहे अर्थातच आम्हाला तो < आवडतो'

In [40]:
for i in range(len(df)):
  df['tweet'][i] = remove_non_marathi_goemotions(df['tweet'][i])
df = df[df['tweet']!='']

<ipython-input-40-8d6e57f33d8a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet'][i] = remove_non_marathi_goemotions(df['tweet'][i])


In [41]:
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
df = df.reset_index(drop=True)

<ipython-input-41-ca293342285f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)
<ipython-input-41-ca293342285f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [45]:
df['tweet'][3054]

'नाम, काय? तू इथेच का आहेस. बंद संभोग.'

In [48]:
from tqdm import tqdm
for i in tqdm(range(len(df))):
    df['english_translated'][i] = translator.translate(df['tweet'][i], src="mr", dest="en").text

  0%|          | 0/12491 [00:00<?, ?it/s]<ipython-input-48-69d591e8be0f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['english_translated'][i] = translator.translate(df['tweet'][i], src="mr", dest="en").text
100%|██████████| 12491/12491 [20:35<00:00, 10.11it/s]


In [49]:
# once dataframe is ready, then create a new dataframe, containing the 'english_translated' and 'label' columns only:
df_containing_english = df[['english_translated', 'label']]
df_containing_english.head()

,english_translated,label
0,"It's so hard to sit through, I can't believe t...",-1
1,Because the idiots in this sub care so much ab...,-1
2,Water is not a human right. That's stupid.,-1
3,The thin sides and contours are really throwin...,-1
4,It will be hard not to get frustrated if there...,-1


In [50]:
df_containing_english['english_translated'][0]

"It's so hard to sit through, I can't believe this dude's swearing, it just makes it all the more unbearable"

In [51]:
df['tweet'][1]

'कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर काय आहे याची खूप काळजी घेतात? हे फक्त मला सांगते की येथे बरेच लोक नार्सिसिस्ट आहेत.'

In [52]:
# now, translate the sentences back to Marathi:
translator.translate(df_containing_english['english_translated'][1], src="en", dest="mr").text

'कारण या उपमधले मूर्ख इतर लोकांच्या स्क्रीनवर काय आहे याची इतकी काळजी घेतात? हे फक्त मला सांगते की येथे बरेच लोक नार्सिसिस्ट आहेत.'

In [53]:
df_backtranslated_english = df_containing_english
df_backtranslated_english.head()

,english_translated,label
0,"It's so hard to sit through, I can't believe t...",-1
1,Because the idiots in this sub care so much ab...,-1
2,Water is not a human right. That's stupid.,-1
3,The thin sides and contours are really throwin...,-1
4,It will be hard not to get frustrated if there...,-1


In [54]:
for i in tqdm(range(len(df_backtranslated_english))):
    df_backtranslated_english['english_translated'][i] = translator.translate(df_backtranslated_english['english_translated'][i], src="en", dest="mr").text

  0%|          | 0/12491 [00:00<?, ?it/s]<ipython-input-54-3a682e511d23>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_backtranslated_english['english_translated'][i] = translator.translate(df_backtranslated_english['english_translated'][i], src="en", dest="mr").text
100%|██████████| 12491/12491 [15:18<00:00, 13.60it/s]


In [55]:
df_backtranslated_english.rename(columns = {'english_translated':'backtranslated'}, inplace = True)

<ipython-input-55-c1d8d77d279b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_backtranslated_english.rename(columns = {'english_translated':'backtranslated'}, inplace = True)


In [56]:
df_backtranslated_english.rename(columns = {'backtranslated':'tweet'}, inplace = True)

<ipython-input-56-10eb77923559>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_backtranslated_english.rename(columns = {'backtranslated':'tweet'}, inplace = True)


In [57]:
df.head()

,tweet,label,english_translated
0,"हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर व...",-1,"It's so hard to sit through, I can't believe t..."
1,कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर...,-1,Because the idiots in this sub care so much ab...
2,पाणी हा मानवी हक्क नाही. ते मूर्खपणाचे आहे.,-1,Water is not a human right. That's stupid.
3,पातळ बाजू आणि अंतरेषा खरोखर मला फेकून देत आहेत.,-1,The thin sides and contours are really throwin...
4,या पॉवरप्लेवर कोणतेही ध्येय नसल्यास निराश न हो...,-1,It will be hard not to get frustrated if there...


In [58]:
df.drop(['english_translated'], axis=1, inplace=True)

In [59]:
df.head()

,tweet,label
0,"हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर व...",-1
1,कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर...,-1
2,पाणी हा मानवी हक्क नाही. ते मूर्खपणाचे आहे.,-1
3,पातळ बाजू आणि अंतरेषा खरोखर मला फेकून देत आहेत.,-1
4,या पॉवरप्लेवर कोणतेही ध्येय नसल्यास निराश न हो...,-1


In [60]:
df_backtranslated_english.head()

,tweet,label
0,"बसणे खूप कठीण आहे, माझा या मुलाच्या शपथेवर विश...",-1
1,कारण या उपमधले मूर्ख इतर लोकांच्या स्क्रीनवर क...,-1
2,पाणी हा मानवी हक्क नाही. ते मूर्ख आहे.,-1
3,पातळ बाजू आणि आकृतिबंध खरोखरच मला फेकून देत आहेत.,-1
4,या पॉवरप्लेवर कोणतेही गोल नसल्यास निराश न होणे...,-1


In [61]:
len(df_backtranslated_english)

12491

In [62]:
df_backtranslated_english.drop_duplicates()

,tweet,label
0,"बसणे खूप कठीण आहे, माझा या मुलाच्या शपथेवर विश...",-1
1,कारण या उपमधले मूर्ख इतर लोकांच्या स्क्रीनवर क...,-1
2,पाणी हा मानवी हक्क नाही. ते मूर्ख आहे.,-1
3,पातळ बाजू आणि आकृतिबंध खरोखरच मला फेकून देत आहेत.,-1
4,या पॉवरप्लेवर कोणतेही गोल नसल्यास निराश न होणे...,-1
...,...,...
12486,स्त्रियांना पुरुषांसारखेच सर्व अधिकार मिळाले प...,1
12487,या व्यक्तीकडे वस्तुस्थिती नंतर काही महान आत्म-...,1
12488,बुडणाऱ्या स्कूल बससारखे मजेदार.,1
12489,सर्वोत्तम ठिकाणे स्थलांतरित रेस्टॉरंट्स आहेत. ...,1


In [63]:
df_backtranslated_english['tweet'][1]

'कारण या उपमधले मूर्ख इतर लोकांच्या स्क्रीनवर काय आहे याची इतकी काळजी घेतात? हे फक्त मला सांगते की येथे बरेच लोक नार्सिसिस्ट आहेत.'

In [ ]:
df['tweet'][1]

In [64]:
df_final = df.append(df_backtranslated_english, ignore_index = True)

<ipython-input-64-6ddf09676330>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df.append(df_backtranslated_english, ignore_index = True)


In [65]:
df_final.to_csv('backtranslated-augmented-without-paraphrasing-goemotions.csv', index=False)
# use this dataset directly for training. do not preprocess this at all.

In [66]:
df_final

,tweet,label
0,"हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर व...",-1
1,कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर...,-1
2,पाणी हा मानवी हक्क नाही. ते मूर्खपणाचे आहे.,-1
3,पातळ बाजू आणि अंतरेषा खरोखर मला फेकून देत आहेत.,-1
4,या पॉवरप्लेवर कोणतेही ध्येय नसल्यास निराश न हो...,-1
...,...,...
24977,स्त्रियांना पुरुषांसारखेच सर्व अधिकार मिळाले प...,1
24978,या व्यक्तीकडे वस्तुस्थिती नंतर काही महान आत्म-...,1
24979,बुडणाऱ्या स्कूल बससारखे मजेदार.,1
24980,सर्वोत्तम ठिकाणे स्थलांतरित रेस्टॉरंट्स आहेत. ...,1


In [70]:
df_final['tweet'][0]

'हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर विश्वास बसत नाही, हे सर्व अधिक असह्य करते'

In [72]:
df_final['tweet'][12491]

'बसणे खूप कठीण आहे, माझा या मुलाच्या शपथेवर विश्वास बसत नाही, हे सर्व अधिक असह्य करते'